## Function to Explore Social Vulnerability Index by Neighborhood

In order to practice the creation of functions, I will be creating a function where the user inputs the name of a neighborhood, and the geographical boundary of the neighborhood along with the range of Social Vulnerability Index (SoVI) scores within the neighborhood will be outputted. 

In [2]:
# Import spatial analysis libraries
import geopandas as gpd
import pandas as pd

# Import a tool that is needed to create a color-coded legend
from branca.colormap import linear

# Import interactive map
import folium

#### The data that will be explored are the neighborhoods in Los Angeles, along with the Social Vulnerability Index scores, which are provided at the Census Tract level.

In [3]:
# Import geodataframe data
nei = gpd.read_file('data/nei.geojson')
sovi = gpd.read_file('data/sovi.geojson')

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


#### In order to compare the geodataframes side by side, the two will be merged using a spatial join.

In [4]:
sovi_by_nei = sovi.sjoin(nei, how="inner", predicate='intersects')

This type of spatial join will include geometries that intersect between boundaries. Therefore a single census tract is allowed to be counted in multiple neighborhood boundaries.

#### It is good practice to make sure the spatial join was successful.

In [5]:
nei.head(3)

,external_i,name,location,latitude,slug_1,sqmi,display_na,set,slug,longitude,name_1,kind,type,geometry
0,acton,Acton,POINT(34.497355239240846 -118.16981019229348),-118.16981019229348,None,39.3391089485,Acton L.A. County Neighborhood (Current),L.A. County Neighborhoods (Current),acton,34.497355239240846,None,L.A. County Neighborhood (Current),unincorporated-area,"MULTIPOLYGON (((-118.20262 34.53899, -118.1894..."
1,adams-normandie,Adams-Normandie,POINT(34.031461499124156 -118.30020800000011),-118.30020800000011,None,0.805350187789,Adams-Normandie L.A. County Neighborhood (Curr...,L.A. County Neighborhoods (Current),adams-normandie,34.031461499124156,None,L.A. County Neighborhood (Current),segment-of-a-city,"MULTIPOLYGON (((-118.30901 34.03741, -118.3004..."
2,agoura-hills,Agoura Hills,POINT(34.146736499122795 -118.75988450000015),-118.75988450000015,None,8.14676029818,Agoura Hills L.A. County Neighborhood (Current),L.A. County Neighborhoods (Current),agoura-hills,34.146736499122795,None,L.A. County Neighborhood (Current),standalone-city,"MULTIPOLYGON (((-118.76193 34.16820, -118.7263..."


Neighborhood are informative because the City of LA is huge. Also, identifying locations by neighborhood avoids confusion between the City of LA and cities within the County of LA.

In [6]:
sovi.head(3)

,Census_Tract,County,CSA_Type,CSA_Label,DRP_Planning_Area,DPH_Service_Planning_Area,City_of_Los_Angeles_CPA,Population,Children,Older_Adults,...,Asian,NHOPI,Other_Race,Two_or_More_Races,SoVI_Score,SoVI_Thirds,ObjectId,Shape__Area,Shape__Length,geometry
0,Census Tract 5709.02,Los Angeles County,City,City of Lakewood,Gateway Planning Area,East,None,3765,27.8,14.0,...,10.9,0.0,0.4,8.0,2.25,3,1,1.536760e+06,5208.283880,"POLYGON ((-118.12509 33.86032, -118.12287 33.8..."
1,Census Tract 5715.02,Los Angeles County,City,City of Long Beach,Gateway Planning Area,South Bay,None,4700,24.6,14.0,...,21.7,0.1,0.6,1.3,-1.77,1,2,2.001416e+06,5938.830961,"POLYGON ((-118.19274 33.84376, -118.18975 33.8..."
2,Census Tract 9106.05,Los Angeles County,City,City of Palmdale,Antelope Valley Planning Area,Antelope Valley,None,4710,38.4,7.0,...,5.3,0.0,0.2,2.5,2.39,3,3,1.996430e+06,5983.046762,"POLYGON ((-118.07616 34.57281, -118.06728 34.5..."


The columns inform which indexes are used to calculate the SoVI score.

#### There will be many columns so make sure all of them can be viewed

In [7]:
# display.max_columns option controls the number of columns to be printed
pd.set_option('display.max_columns', None)

In [8]:
sovi_by_nei.sample(3)

,Census_Tract,County,CSA_Type,CSA_Label,DRP_Planning_Area,DPH_Service_Planning_Area,City_of_Los_Angeles_CPA,Population,Children,Older_Adults,Older_Adults_Living_Alone,Limited_English,No_High_School_Diploma,Female,Female_Householder,Asthma,Cardiovascular_Disease,Disability,No_Health_Insurance,Living_in_Group_Quarters,Mobile_Homes,Rent_Burden,Renters,Median_Income,Poverty,Households_Without_Vehicle_Acce,Transit_Access,Outdoor_Workers,Unemployed,Foreign_Born,Library_Access,No_Internet_Subscription,Voter_Turnout_Rate,Hispanic_Latinx,White,Black,Native_Tribal,Asian,NHOPI,Other_Race,Two_or_More_Races,SoVI_Score,SoVI_Thirds,ObjectId,Shape__Area,Shape__Length,geometry,index_right,external_i,name,location,latitude,slug_1,sqmi,display_na,set,slug,longitude,name_1,kind,type
514,Census Tract 2132.01,Los Angeles County,City,Los Angeles - Koreatown,Metro Planning Area,Metro,Wilshire,4330,29.9,13.2,9.4,45.2,31.3,53.0,17.8,56.7,7.0,6.2,23.2,0.0,1.7,72.7,93.5,30335,33.6,22.9,100.0,8.2,7.3,62.0,0.6,27.4,52.0,48.7,2.7,1.4,0.0,46.5,0.0,0.0,0.6,5.00,3,515,4.870262e+05,3011.561826,"POLYGON ((-118.30906 34.05444, -118.30742 34.0...",28,koreatown,Koreatown,POINT(34.064510499123763 -118.3049585),-118.3049585,None,2.70048311541,Koreatown L.A. County Neighborhood (Current),L.A. County Neighborhoods (Current),koreatown,34.064510499123763,None,L.A. County Neighborhood (Current),segment-of-a-city
945,Census Tract 9102.06,Los Angeles County,City,City of Palmdale,Antelope Valley Planning Area,Antelope Valley,None,3608,32.5,6.7,3.2,6.3,10.1,51.8,13.0,42.4,10.5,9.2,6.9,0.0,0.0,38.8,16.6,105170,8.0,0.7,0.0,3.1,4.9,26.6,4.7,3.0,68.0,35.3,32.8,13.5,0.6,12.3,0.0,0.9,4.6,-0.38,2,946,7.471815e+07,45064.787828,"POLYGON ((-118.28889 34.58804, -118.28804 34.5...",215,southeast-antelope-valley,Southeast Antelope Valley,POINT(34.466085813776999 -117.92981254313324),-117.92981254313324,None,192.960465064,Southeast Antelope Valley L.A. County Neighbor...,L.A. County Neighborhoods (Current),southeast-antelope-valley,34.466085813776999,None,L.A. County Neighborhood (Current),unincorporated-area
775,Census Tract 1237,Los Angeles County,City,Los Angeles - North Hollywood,San Fernando Valley Planning Area,San Fernando,North Hollywood - Valley Village,4338,21.2,16.0,12.0,14.5,20.5,49.1,5.8,50.2,9.4,11.7,8.2,0.2,0.0,59.0,43.5,66726,10.7,12.5,26.8,6.5,4.6,45.9,0.8,25.4,59.8,31.0,60.4,2.8,0.6,3.1,0.0,0.0,2.1,2.87,3,776,1.395182e+06,4836.786354,"POLYGON ((-118.41379 34.19031, -118.40995 34.1...",239,valley-glen,Valley Glen,POINT(34.189504499122229 -118.4145170000001),-118.4145170000001,None,4.80554447147,Valley Glen L.A. County Neighborhood (Current),L.A. County Neighborhoods (Current),valley-glen,34.189504499122229,None,L.A. County Neighborhood (Current),segment-of-a-city


The Census Tracts have been matched to a neighborhood. Now we can proceed.

#### First, let's look at the SoVI score distribution across LA (not by the neighborhood scale)

In [ ]:
# Creation the folium map
m = folium.Map(location=[34.2, -118.2], # Center around LA
                zoom_start=9,
                tiles='https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png', # Add a desired basemap 
                attr='My Data Attribution')
     
# Define the map
fig = folium.Choropleth(
    geo_data=sovi_by_nei, # Specify what geodataframe is being used
    name='Social Vulnerability Index',
    data=sovi_by_nei,
    columns=['name', 'SoVI_Score'], # The SoVI score will be mapped
    key_on='feature.properties.name',
    fill_color='Purples',
    line_weight=0.1,
    fill_opacity=0.6,
    line_opacity=0.2,
    legend_name='Social Vulnerability Index',
    bins=9,
    reset=True,
    # The scale of the legend is bounded by the smalled and largest given SoVI score
    colormap=linear.Purples_09.scale(
        sovi_by_nei['SoVI_Score'].min(),
        sovi_by_nei['SoVI_Score'].max())
        )
        
# Add the choropleth to the folium map
fig.add_to(m)

# Display map
m

One can observe that the highest social vulnerability is centered around Downtown LA, Van Nuys, and Antelope Valley.

## Let's add interactive components to the map

In [ ]:
# The style function will be empty so that it does not compete with the existing color scheme
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}

# Highlight the census tract when hovered over
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

# Informs user when hovering over the map
tip = folium.features.GeoJson(
    sovi_by_nei,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name','SoVI_Score','Census_Tract'],
        aliases=['Neighborhood: ','SoVI', 'Census Tract'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(tip)
# The parameters in tip should always be in front of this layer to allow for interactivity
m.keep_in_front(tip)
folium.LayerControl().add_to(m)

Now, the user can easily match the Census Tract with the neighborhood.

## However, it is a lot of data looking at the entire LA County. Now a function will be created to observed specific neighborhoods.

In [ ]:
def nei_map(name):
        # This is the basic creation of a folium map that sets the starting location and zoom level and basemap.
    m = folium.Map(location=[34.2, -118.2],
                   zoom_start=9,
                   tiles='https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png',
                   attr='My Data Attribution',
                   tooltip='geojson')
       
    fig = folium.Choropleth(
        geo_data=sovi_by_nei[sovi_by_nei['name']==name],
        name='Social Vulnerability Index for '+ name,
        data=sovi_by_nei[sovi_by_nei['name']==name],
        columns=['name', 'SoVI_Score'], # The SoVI score will be mapped
        key_on='feature.properties.name',
        fill_color='Purples',
        line_weight=0.1,
        fill_opacity=0.6,
        line_opacity=0.2,
        legend_name='Social Vulnerability Index',
        bins=9,
        reset=True,
        # The scale of the legend is bounded by the smalled and largest given SoVI score
        colormap=linear.Purples_09.scale(
            sovi_by_nei['SoVI_Score'].min(),
            sovi_by_nei['SoVI_Score'].max())
            )
    
    # The style function will be empty so that it does not compete with the existing color scheme
    style_function = lambda x: {'fillColor': '#ffffff', 
                                'color':'#000000', 
                                'fillOpacity': 0.1, 
                                'weight': 0.1}

    # Highlight the census tract when hovered over
    highlight_function = lambda x: {'fillColor': '#000000', 
                                    'color':'#000000', 
                                    'fillOpacity': 0.50, 
                                    'weight': 0.1}

    # Informs user when hovering over the map
    tip = folium.features.GeoJson(
        sovi_by_nei,
        style_function=style_function, 
        control=False,
        highlight_function=highlight_function, 
        tooltip=folium.features.GeoJsonTooltip(
            fields=['name','SoVI_Score','Census_Tract'],
            aliases=['Neighborhood: ','SoVI', 'Census Tract'],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    )
    # Add first layer
    fig.add_to(m) 
    
    # Add interactive components
    m.add_child(tip)
    # The parameters in tip should always be in front of this layer to allow for interactivity
    m.keep_in_front(tip)
    folium.LayerControl().add_to(m)

    # return the map at the end of the function to actually execute the code.
    return m
    

## Now, we can look at specific neighborhoods

The following list is a list of all neighborhoods. Caution: the user must input the neighborhood name in the same manner as is presented in the list.

In [ ]:
# Get a list of all the neighborhoods
nei_list = sovi_by_nei.name.unique().tolist()
nei_list

Now, we can see which Census Tracts make up the neighborhood of Westwood. In Westwood, SoVI ranges from -6.8 to -5.8. This means that they are not a socially vulnerable population. 

In [ ]:
nei_map('Van Nuys')

As previously noticed, Van Nuys has a high range of SoVI (6.0 to 7.0).